In [1]:
import requests
import grab
import json
import logging
import codecs
from lxml.etree import ElementTree, fromstring
from bs4 import BeautifulSoup
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
from IPython.display import HTML

### Get All Movie Links

In [2]:
url = 'https://www.kinopoisk.ru/top/lists/186/filtr/all/sort/order/page/%d/perpage/200/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko)' + 
                  'Chrome/61.0.3163.100 Safari/537.36',
}
g = grab.Grab(headers=headers)

MOVIE_URLS = set()

for x in [1,2,3,4,5]:
    try:
        g.go(url % x)
        MOVIE_URLS.update([x for x in g.xpath_list('//a/@href') if x.startswith('/film/') and x.count('/')==3])
    except Exception as e:
        logging.error(str(e))
        logging.error('Error on %s' % x)

MOVIE_URLS = list(MOVIE_URLS)
print 'Total movie urls: %d' % len(MOVIE_URLS)
print 'Examples: \n%s' % '\n'.join(MOVIE_URLS[:5])

/Users/vladfedorenko/anaconda/lib/python2.7/site-packages/grab/util/warning.py:28: GrabDeprecationWarning: Call to deprecated function xpath_list. Use grab.doc.select() instead.
  warn(message)


Total movie urls: 1039
Examples: 
/film/neuderzhimye-2010-432550/
/film/neveroyatnyy-khalk-2008-255380/
/film/komnata-strakha-2002-780/
/film/ekzamen-2009-418762/
/film/shpionskiy-most-2015-840884/


In [3]:
%%time
TOTAL_REVIEWS = 0
with codecs.open('./data.csv', mode='w', encoding='utf-8') as f_out:
    for x in tqdm(MOVIE_URLS[:10]):
        name = x.split('/')[2]
        try:
            i = 1
            while True:
                g.go('https://www.kinopoisk.ru%sord/rating/perpage/200/page/%d/#list' % (x, i), timeout=30)
                reviews = g.xpath_list("//div[@class='reviewItem userReview']")
                if not reviews:
                    break
                for y in reviews:
                    d = {'name': name}
                    if 'bad' in y.getchildren()[2].get('class', ''):
                        d['sentiment'] = 'negative'
                    elif 'good' in y.getchildren()[2].get('class', ''):
                        d['sentiment'] = 'positive'
                    else:
                        d['sentiment'] = 'neutral'
                    d['text'] = y.getchildren()[2].getchildren()[2].text_content().replace('\n', ' ')\
                        .replace('\t', ' ').replace('\r', ' ').strip()
                    f_out.write('%s\t%s\t%s\n' % (d['name'], d['sentiment'], d['text']))
                    TOTAL_REVIEWS += 1
                else:
                    break
                i += 1
        except Exception as e:
            logging.error('Error on %s: %s' % (name, str(e)))
print 'Total reviews: %d' % TOTAL_REVIEWS

100%|██████████| 10/10 [02:04<00:00, 13.16s/it]

Total reviews: 1759
CPU times: user 1.25 s, sys: 106 ms, total: 1.35 s
Wall time: 2min 4s


### Too slow...